Data Profiling – Primeira Análise de Qualidade da Base

In [0]:
# Importação de bibliotecas
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Definição do caminho da pasta Bronze
catalogo = "medalhao"
silver_db_name = "silver"

# Leitura das tabelas Bronze
categoria_produtos_bronze_df = spark.table("medalhao.bronze.dm_categoria_produtos_traducao")
cotacao_dolar_bronze_df = spark.table("medalhao.bronze.dm_cotacao_dolar")
avaliacoes_pedidos_bronze_df = spark.table("medalhao.bronze.ft_avaliacoes_pedidos")
consumidores_bronze_df = spark.table("medalhao.bronze.ft_consumidores")
geolocalizacao_bronze_df = spark.table("medalhao.bronze.ft_geolocalizacao")
itens_pedidos_bronze_df = spark.table("medalhao.bronze.ft_itens_pedidos")
pagamentos_pedidos_bronze_df = spark.table("medalhao.bronze.ft_pagamentos_pedidos")
pedidos_bronze_df = spark.table("medalhao.bronze.ft_pedidos")
produtos_bronze_df = spark.table("medalhao.bronze.ft_produtos")
vendedores_bronze_df = spark.table("medalhao.bronze.ft_vendedores")

lista_df = [categoria_produtos_bronze_df, cotacao_dolar_bronze_df, avaliacoes_pedidos_bronze_df, consumidores_bronze_df, geolocalizacao_bronze_df, itens_pedidos_bronze_df, pagamentos_pedidos_bronze_df, pedidos_bronze_df, produtos_bronze_df, vendedores_bronze_df]


In [0]:
spark.sql("DROP TABLE IF EXISTS medalhao.silver.categoria_produtos_silver")
spark.sql("DROP TABLE IF EXISTS medalhao.silver.avaliacoes_pedidos_silver")
spark.sql("DROP TABLE IF EXISTS medalhao.silver.consumidores_silver")
spark.sql("DROP TABLE IF EXISTS medalhao.silver.itens_pedidos_silver")
spark.sql("DROP TABLE IF EXISTS medalhao.silver.pagamentos_pedidos_silver")
spark.sql("DROP TABLE IF EXISTS medalhao.silver.pedidos_silver")
spark.sql("DROP TABLE IF EXISTS medalhao.silver.produtos_silver")
spark.sql("DROP TABLE IF EXISTS medalhao.silver.vendedores_silver")



Checando quais são os DF que possuem nulos

In [0]:
'''
for i in lista_df:
    nulos_df = i.select([
        F.count(F.when(F.col(c).isNull(), c)).alias(c)
        for c in i.columns
    ]).display()
'''


In [0]:
df_com_nulos = [categoria_produtos_bronze_df, avaliacoes_pedidos_bronze_df,pedidos_bronze_df,produtos_bronze_df]

In [0]:
consumidores_silver_df = (
    consumidores_bronze_df
    .select(
        F.col("customer_id").alias("id_consumidor"),
        F.col("customer_zip_code_prefix").alias("prefixo_cep"),
        F.upper(F.col("customer_city")).alias("cidade"),
        F.upper(F.col("customer_state")).alias("estado"),
    )
    .dropDuplicates(["id_consumidor"])
    .withColumn("data_ingestao", F.current_timestamp())
)
consumidores_silver_df.write.format("delta").mode("overwrite").saveAsTable(f"{catalogo}.{silver_db_name}.consumidores_silver_df")
consumidores_silver_df.display()
print("✅ Tabela silver.ft_consumidores criada com sucesso!\\n")

In [0]:
pedidos_silver_df = (
    pedidos_bronze_df
    .select(
        F.col("order_id").alias("id_pedido"),
        F.col("customer_id").alias("id_consumidor"),
        F.col("order_status").alias("status"),
        F.to_date("order_purchase_timestamp").alias("pedido_compra_timestamp"),
        F.to_date("order_approved_at").alias("pedido_aprovado_timestamp"),
        F.to_date("order_delivered_carrier_date").alias("pedido_carregado_timestamp"),
        F.to_date("order_delivered_customer_date").alias("pedido_entregue_timestamp"),
        F.to_date("order_estimated_delivery_date").alias("pedido_estimativa_entrega_timestamp"),
    )
    .withColumn("data_ingestao", F.current_timestamp())
    .withColumn(
        "status",
        F.when(F.col("status") == "delivered", "entregue")
        .when(F.col("status") == "canceled", "cancelado")
        .when(F.col("status") == "invoiced", "faturado")
        .when(F.col("status") == "processing", "em processamento")
        .when(F.col("status") == "shipped", "enviado")
        .when(F.col("status") == "unavailable", "indisponível")
        .when(F.col("status") == "created", "criado")
        .when(F.col("status") == "approved", "aprovado")
    )
    .withColumn(
        "tempo_entrega_dias",
        F.datediff(F.col("pedido_entregue_timestamp"), F.col("pedido_compra_timestamp"))
    )
    .withColumn(
        "tempo_entrega_estimado_dias",
        F.datediff(F.col("pedido_estimativa_entrega_timestamp"), F.col("pedido_compra_timestamp"))
    )
    .withColumn(
        "diferenca_entrega_dias",
        F.datediff(F.col("pedido_entregue_timestamp"), F.col("pedido_estimativa_entrega_timestamp"))
    )
    .withColumn(
        "entrega_no_prazo",
        F.when(F.col("pedido_entregue_timestamp").isNull(), "Não Entregue")  
        .when(F.col("diferenca_entrega_dias") <= 0, "Sim")                 
        .when(F.col("diferenca_entrega_dias") > 0, "Não") 
    )
)
pedidos_silver_df.write.format("delta").mode("overwrite").saveAsTable(f"{catalogo}.{silver_db_name}.pedidos_silver")
pedidos_silver_df.display()
print("✅ Tabela silver.pedidos_silver criada com sucesso!\n")
#PRECISO TRATAR OS VALORES NULO
#fazer um groupby count para ver quantos tem entregues, em processamento e etc...

In [0]:
nulos_df = pedidos_silver_df.select([
        F.count(F.when(F.col(c).isNull(), c)).alias(c)
        for c in pedidos_silver_df.columns
    ]).display()
pedidos_silver_df


pedidos_silver_df.groupBy("status").count().display()


In [0]:
'''
%sql
ALTER TABLE medalhao.silver.pedidos_silver
ADD COLUMNS (
  tempo_entrega_dias INT,
  tempo_entrega_estimado_dias INT,
  diferenca_entrega_dias INT,
  entrega_no_prazo STRING
)
COLOCAR TODOS OS COMANDOS SQL QUE PUDER AQUI
'''

In [0]:
itens_pedidos_silver_df = (
    itens_pedidos_bronze_df.select(
        F.col("order_id").alias("id_pedido"),
        F.col("order_item_id").alias("id_item"),
        F.col("product_id").alias("id_produto"),
        F.col("seller_id").alias("id_vendedor"),
        F.col("price").alias("preco_BRL"),
        F.col("freight_value").alias("preco_frete"),
    )
    .withColumn("data_ingestao", F.current_timestamp())
    .withColumn("preço_total", F.round((F.col("preco_BRL") + F.col("preco_frete")), 2))
)
itens_pedidos_silver_df.write.format("delta").mode("overwrite").saveAsTable(f"{catalogo}.{silver_db_name}.itens_pedidos_silver")
itens_pedidos_silver_df.display()
print("✅ Tabela silver.itens_pedidos_silver criada com sucesso!\n")

In [0]:
pagamentos_pedidos_silver_df = (
    pagamentos_pedidos_bronze_df.select(
        F.col("order_id").alias("id_pedido"),
        F.col("payment_sequential").alias("codigo_pagamento"),
        F.col("payment_type").alias("forma_pagamento"),
        F.col("payment_installments").alias("parcelas"),
        F.col("payment_value").alias("valor_pagamento")
    )
    .withColumn("forma_pagamento",
    F.when(F.col("forma_pagamento") == "credit_card","Cartão de Crédito")
    .when(F.col("forma_pagamento") == "boleto","Boleto")
    .when(F.col("forma_pagamento") == "voucher","Voucher")
    .when(F.col("forma_pagamento") == "debit_card","Cartão de Débito")
    .otherwise("Outros")
    )
    .withColumn("data_ingestao", F.current_timestamp())

)
pagamentos_pedidos_silver_df.write.format("delta").mode("overwrite").saveAsTable(f"{catalogo}.{silver_db_name}.pagamentos_pedidos_silver")
pagamentos_pedidos_silver_df.display()

In [0]:
pagamentos_pedidos_silver_df.groupBy("forma_pagamento").count().display()


In [0]:
avaliacoes_pedidos_silver_df = (
    avaliacoes_pedidos_bronze_df.select(
        F.col("review_id").alias("id_avaliacao"),
        F.col("order_id").alias("id_pedido"),
        F.col("review_score").alias("avaliacao"),
        F.col("review_comment_title").alias("titulo_comentario"),
        F.col("review_comment_message").alias("comentario"),
        F.to_date("review_creation_date").alias("data_comentario"),
        F.to_date("review_answer_timestamp").alias("data_resposta")
    )
    .withColumn("data_ingestao", F.current_timestamp())
    .filter(F.col("id_pedido").isNotNull())
    .filter(F.length(F.col("id_avaliacao")) == 32)
    .filter(F.length(F.col("id_pedido")) == 32)
    .filter(~F.col("id_avaliacao").rlike("[^a-zA-Z0-9]"))
    .filter(~F.col("id_pedido").rlike("[^a-zA-Z0-9]"))
    .filter(~F.col("avaliacao").rlike("[^0-5]"))
    .filter(F.col("data_comentario").isNotNull())
    .filter(F.datediff("data_comentario", "data_ingestao")<0)

)
avaliacoes_pedidos_silver_df.write.format("delta").mode("overwrite").saveAsTable(f"{catalogo}.{silver_db_name}.avaliacoes_pedidos_silver")

avaliacoes_pedidos_silver_df.display()

In [0]:
avaliacoes_pedidos_silver_df.groupBy("avaliacao").count().display()


In [0]:
produtos_silver_df = (
    produtos_bronze_df.select(
        F.col("product_id").alias("id_produto"),
        F.coalesce("product_category_name", F.lit("outros")).alias("categoria_produto"),
        F.coalesce("product_weight_g", F.lit("0")).alias("peso_produto_gramas"),
        F.coalesce("product_length_cm", F.lit("0")).alias("comprimento_centimetros"),
        F.coalesce("product_height_cm", F.lit("0")).alias("altura_centimetros"),
        F.coalesce("product_width_cm", F.lit("0")).alias("largura_centimetros")

    )
    .withColumn("data_ingestao", F.current_timestamp())

)
produtos_silver_df.write.format("delta").mode("overwrite").saveAsTable(f"{catalogo}.{silver_db_name}.produtos_silver")
produtos_silver_df.display()
print("✅ Tabela silver.produtos_silver criada com sucesso!\n")

In [0]:
vendedores_silver_df = (
    vendedores_bronze_df.select(
        F.col("seller_id").alias("id_vendedor"),
        F.col("seller_zip_code_prefix").alias("cep_vendedor"),
        F.upper(F.col("seller_city")).alias("cidade"),
        F.upper(F.col("seller_state")).alias("estado")
    )
    .withColumn("data_ingestao", F.current_timestamp())

)
vendedores_silver_df.write.format("delta").mode("overwrite").saveAsTable(f"{catalogo}.{silver_db_name}.vendedores_silver")
vendedores_silver_df.display()
print("✅ Tabela silver.vendedores_silver criada com sucesso!\n")

In [0]:
categoria_produtos_silver_df = (
    categoria_produtos_bronze_df.select(
        F.col("product_category_name").alias("nome_produto_pt"),
        F.col("product_category_name_english").alias("nome_produto_en")
    )
    .withColumn("data_ingestao", F.current_timestamp())
    #lembrar dropduplicates
    #lembrar groupby por produto para checar se tem duplicadas
)
categoria_produtos_silver_df.write.format("delta").mode("overwrite").saveAsTable(f"{catalogo}.{silver_db_name}.categoria_produtos_silver")
categoria_produtos_silver_df.display()
print("✅ Tabela silver.categoria_produtos_silver criada com sucesso!\n")


In [0]:
from pyspark.sql.window import Window
from datetime import timedelta

# 1. Preparar dados da Bronze
cotacao_dolar_temp = (
    cotacao_dolar_bronze_df
    .select(
        F.to_date("dataHoraCotacao").alias("data"),
        F.col("cotacaoCompra").cast("decimal(10,4)").alias("cotacao_dolar")
    )
)

# 2. Obter período
min_max = cotacao_dolar_temp.agg(
    F.min("data").alias("data_min"),
    F.max("data").alias("data_max")
).collect()[0]

# 3. Gerar TODAS as datas
todas_datas = []
data_atual = min_max["data_min"]
while data_atual <= min_max["data_max"]:
    todas_datas.append((data_atual,))
    data_atual += timedelta(days=1)

df_todas_datas = spark.createDataFrame(todas_datas, ["data"])

# 4. LEFT JOIN (fins de semana ficam NULL)
cotacao_completa = df_todas_datas.join(cotacao_dolar_temp, "data", "left")

# 5. WINDOW FUNCTION: Preencher NULLs com último valor disponível
window_spec = Window.orderBy("data").rowsBetween(Window.unboundedPreceding, Window.currentRow)

cotacao_dolar_silver_df = cotacao_completa.withColumn(
    "cotacao_dolar",
    F.last("cotacao_dolar", ignorenulls=True).over(window_spec)
)

# 6. Adicionar timestamp
cotacao_dolar_silver_df = cotacao_dolar_silver_df.withColumn("ingestion_timestamp", F.current_timestamp())

# 7. Salvar
cotacao_dolar_silver_df.write.format("delta").mode("overwrite").saveAsTable(f"{catalogo}.{silver_db_name}.dm_cotacao_dolar")

print("✅ Tabela silver.dm_cotacao_dolar criada com sucesso!\n")
cotacao_dolar_silver_df.orderBy("data").display()

In [0]:
from pyspark.sql import functions as F

#Validação de integridade referencial

#Carregando as tabelas de pedidos, consumidores e itens
pedidos_silver_df = spark.table(f"{catalogo}.{silver_db_name}.pedidos_silver")
consumidores_silver_df = spark.table(f"{catalogo}.{silver_db_name}.consumidores_silver_df")
itens_pedidos_silver_df = spark.table(f"{catalogo}.{silver_db_name}.itens_pedidos_silver")

#Verificação de Pedidos Órfãos com consumidor

pedidos_orfaos_df = pedidos_silver_df.join(
    consumidores_silver_df,
    on=["id_consumidor"], 
    how="left_anti"        
)

count_pedidos_orfaos = pedidos_orfaos_df.count()
print(f"Total de PEDIDOS órfãos encontrados (sem consumidor): {count_pedidos_orfaos}")

#Verificação de Itens Órfãos com pedido
itens_orfaos_df = itens_pedidos_silver_df.join(
    pedidos_silver_df,
    on=["id_pedido"],   
    how="left_anti"      
)

count_itens_orfaos = itens_orfaos_df.count()
print(f"Total de ITENS órfãos encontrados (sem pedido): {count_itens_orfaos}")

print("Não há pedidos orfãos")

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import DecimalType

print("Iniciando a criação da tabela final silver.pedido_total...")

# Definições (caso não estejam em outra célula)
catalogo = "medalhao"
silver_db_name = "silver"

# --- 1. Agregar Pagamentos ---
# Carrega a tabela de pagamentos da camada Silver
pagamentos_silver_df = spark.table(f"{catalogo}.{silver_db_name}.pagamentos_pedidos_silver")

# Agrupa por 'id_pedido' e soma o 'valor_pagamento'
pagamentos_agg_df = pagamentos_silver_df.groupBy("id_pedido").agg(
    F.round(F.sum("valor_pagamento"), 2).alias("valor_total_pago_brl")
)

# --- 2. Carregar as outras fontes Silver (ESTA PARTE ESTAVA FALTANDO) ---
pedidos_df = spark.table(f"{catalogo}.{silver_db_name}.pedidos_silver")
consumidores_df = spark.table(f"{catalogo}.{silver_db_name}.consumidores_silver_df")
cotacao_df = spark.table(f"{catalogo}.{silver_db_name}.cotacao_dolar_silver")

# --- 3. Juntar as Fontes ---
# Começamos com Pedidos e juntamos Consumidores
base_df = pedidos_df.join(
    consumidores_df,
    on=["id_consumidor"],
    how="inner" 
)

# Juntamos os pagamentos agregados
base_com_pagamento_df = base_df.join(
    pagamentos_agg_df,
    on=["id_pedido"],
    how="inner" 
)

# Juntamos a cotação do dólar
# ATENÇÃO: As colunas de data têm nomes diferentes!
base_com_cotacao_df = base_com_pagamento_df.join(
    cotacao_df,
    # Chave do Join: data da compra == data da cotação
    base_com_pagamento_df.pedido_compra_timestamp == cotacao_df.data, 
    how="left" 
)

# --- 4. Calcular e Selecionar Colunas Finais ---
pedido_total_df = base_com_cotacao_df.select(
    F.col("id_pedido"),
    F.col("id_consumidor"),
    F.col("status"),
    F.col("pedido_compra_timestamp").alias("data_pedido"),
    F.col("valor_total_pago_brl"),
    
    # Calcular o valor em USD
    F.round((F.col("valor_total_pago_brl") / F.col("cotacao_dolar")), 2).alias("valor_total_pago_usd")
    
).fillna(0, subset=["valor_total_pago_usd"]) # Trata casos onde a cotação era nula

# --- 5. Salvar a Tabela Final ---
pedido_total_df.write.format("delta").mode("overwrite").saveAsTable(f"{catalogo}.{silver_db_name}.pedido_total")

print("✅ Tabela final silver.pedido_total criada com sucesso!\n")
pedido_total_df.display()